In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=8d5214ddd2e68095daf4eedba4e2663778a6b90bdbdab5d9f51a755e35b40238
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [3]:
spark  = SparkSession.builder.appName("test").getOrCreate()

In [24]:
# Create DataFrame
df=spark.createDataFrame([Row(Area='Urban',Land=None,Soil='Black',
Field_count=100,Crop_Name='Paddy',Quantity=34000),
                                                  Row(Area='Urban',Land='Rich',Soil='Red',Field_count=20,Crop_Name='Wheat',
Quantity=200000),
                                                  Row(Area='Rural',Land='Poor',Soil=None,Field_count=None,Crop_Name='Paddy',
Quantity=24000),
                                                  Row(Area=None,Land='Poor',Soil='Black',Field_count=400,Crop_Name='Paddy',
Quantity=None),
                                                  Row(Area=None,Land='Poor',Soil='Red',Field_count=None,Crop_Name=float("NaN"),
Quantity=None)
                                                  ])

df.show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [16]:
df.select(df.Area,isnull(df.Area)).show()

+-----+--------------+
| Area|(Area IS NULL)|
+-----+--------------+
|Urban|         false|
|Urban|         false|
|Rural|         false|
| NULL|          true|
| NULL|          true|
+-----+--------------+



In [17]:
df.select(df.Crop_Name,isnan(df.Crop_Name)).show()

+---------+----------------+
|Crop_Name|isnan(Crop_Name)|
+---------+----------------+
|    Paddy|           false|
|    Wheat|           false|
|    Paddy|           false|
|    Paddy|           false|
|      NaN|            true|
+---------+----------------+



In [25]:
#nanvl(col1,col2): Returns the first column that is not null
df.select(df.Field_count,df.Crop_Name,nanvl(df.Field_count,df.Crop_Name)).show()

+-----------+---------+-----------------------------+
|Field_count|Crop_Name|nanvl(Field_count, Crop_Name)|
+-----------+---------+-----------------------------+
|        100|    Paddy|                        100.0|
|         20|    Wheat|                         20.0|
|       NULL|    Paddy|                         NULL|
|        400|    Paddy|                        400.0|
|       NULL|      NaN|                         NULL|
+-----------+---------+-----------------------------+



In [27]:
df.select(df.Field_count,df.Crop_Name,coalesce(df.Field_count,df.Crop_Name)).show()

+-----------+---------+--------------------------------+
|Field_count|Crop_Name|coalesce(Field_count, Crop_Name)|
+-----------+---------+--------------------------------+
|        100|    Paddy|                             100|
|         20|    Wheat|                              20|
|       NULL|    Paddy|                           Paddy|
|        400|    Paddy|                             400|
|       NULL|      NaN|                             NaN|
+-----------+---------+--------------------------------+



In [28]:
df.select(df.Crop_Name,df.Land,coalesce(df.Crop_Name,df.Land)).show()

+---------+----+-------------------------+
|Crop_Name|Land|coalesce(Crop_Name, Land)|
+---------+----+-------------------------+
|    Paddy|NULL|                    Paddy|
|    Wheat|Rich|                    Wheat|
|    Paddy|Poor|                    Paddy|
|    Paddy|Poor|                    Paddy|
|      NaN|Poor|                      NaN|
+---------+----+-------------------------+



In [29]:
df.na.drop().show()

+-----+----+----+-----------+---------+--------+
| Area|Land|Soil|Field_count|Crop_Name|Quantity|
+-----+----+----+-----------+---------+--------+
|Urban|Rich| Red|         20|    Wheat|  200000|
+-----+----+----+-----------+---------+--------+



In [31]:
#drops rows only when all row elements are nulls
df.na.drop(how='all').show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [37]:
#There should be a not null values of minimum 1
df.na.drop(thresh=1).show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [33]:
df.na.drop(thresh=2).show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [34]:
df.na.drop(thresh=3).show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [38]:
#There should be a not null values of minimum 4
df.na.drop(thresh=4).show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
+-----+----+-----+-----------+---------+--------+



In [41]:
#There should be a not null values of minimum 5
df.na.drop(thresh=5).show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
+-----+----+-----+-----------+---------+--------+



In [40]:
#There should be a not null values of minimum 6
df.na.drop(thresh=6).show()

+-----+----+----+-----------+---------+--------+
| Area|Land|Soil|Field_count|Crop_Name|Quantity|
+-----+----+----+-----------+---------+--------+
|Urban|Rich| Red|         20|    Wheat|  200000|
+-----+----+----+-----------+---------+--------+



In [42]:
#There should be a not null values of minimum 7
df.na.drop(thresh=7).show()

+----+----+----+-----------+---------+--------+
|Area|Land|Soil|Field_count|Crop_Name|Quantity|
+----+----+----+-----------+---------+--------+
+----+----+----+-----------+---------+--------+



In [43]:
#drop null applied to specific column
df.na.drop(subset = "Crop_Name").show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [46]:
#drop null applied to specific column
df.na.drop(subset ="Area").show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
+-----+----+-----+-----------+---------+--------+



In [48]:
df.show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|       NULL|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|       NULL|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [49]:
df.na.fill(50).show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|         50|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|      50|
| NULL|Poor|  Red|         50|      NaN|      50|
+-----+----+-----+-----------+---------+--------+



In [52]:
#fill na in Specific column
df.na.fill(50,"Field_count").show()

+-----+----+-----+-----------+---------+--------+
| Area|Land| Soil|Field_count|Crop_Name|Quantity|
+-----+----+-----+-----------+---------+--------+
|Urban|NULL|Black|        100|    Paddy|   34000|
|Urban|Rich|  Red|         20|    Wheat|  200000|
|Rural|Poor| NULL|         50|    Paddy|   24000|
| NULL|Poor|Black|        400|    Paddy|    NULL|
| NULL|Poor|  Red|         50|      NaN|    NULL|
+-----+----+-----+-----------+---------+--------+



In [56]:
#replace
df.na.fill('yellow',"Soil").show()

+-----+----+------+-----------+---------+--------+
| Area|Land|  Soil|Field_count|Crop_Name|Quantity|
+-----+----+------+-----------+---------+--------+
|Urban|NULL| Black|        100|    Paddy|   34000|
|Urban|Rich|   Red|         20|    Wheat|  200000|
|Rural|Poor|yellow|       NULL|    Paddy|   24000|
| NULL|Poor| Black|        400|    Paddy|    NULL|
| NULL|Poor|   Red|       NULL|      NaN|    NULL|
+-----+----+------+-----------+---------+--------+



In [53]:
#replace
soil = {'Black':'Brown','Red':'Yellow'}

df.na.replace(soil,"Soil").show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:4667: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


+-----+----+------+-----------+---------+--------+
| Area|Land|  Soil|Field_count|Crop_Name|Quantity|
+-----+----+------+-----------+---------+--------+
|Urban|NULL| Brown|        100|    Paddy|   34000|
|Urban|Rich|Yellow|         20|    Wheat|  200000|
|Rural|Poor|  NULL|       NULL|    Paddy|   24000|
| NULL|Poor| Brown|        400|    Paddy|    NULL|
| NULL|Poor|Yellow|       NULL|      NaN|    NULL|
+-----+----+------+-----------+---------+--------+

